In [1]:
import numpy as np

In [8]:
fit_dummy = np.random.rand(100, 3)
v = fit_dummy.T @ (fit_dummy - np.mean(fit_dummy, axis=0))

In [9]:
v2 = fit_dummy.T @ fit_dummy - np.sum(fit_dummy.T, axis=1, keepdims=True) * np.mean(fit_dummy, axis=0, keepdims=True)

In [11]:
v3 = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        v3[i, j] = np.sum(fit_dummy[:, j] * (fit_dummy[:, i] - np.mean(fit_dummy[:, i])))

In [12]:
np.allclose(v, v3)

True

In [2]:
t = np.zeros((3, 3))
t - np.ones((3, 1))

array([[-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.]])

In [3]:
np.ones((3, 1)).shape

(3, 1)

In [4]:
np.ones((3, 1))

array([[1.],
       [1.],
       [1.]])

In [30]:
test_mat = np.random.rand(3,3)
test_mat = test_mat.T @ test_mat
test_mat

array([[0.27661964, 0.44851491, 0.46945992],
       [0.44851491, 1.27801924, 0.90478566],
       [0.46945992, 0.90478566, 0.83420293]])

In [37]:
test_vec = np.random.rand(3)
test_vec[-1] += 5
test_vec

array([0.3899092 , 0.28447892, 5.78215143])

In [38]:
add_mat = np.zeros((3, 3))
add_mat[:, -1] = test_vec
add_mat[-1, :] = test_vec
add_mat

array([[0.        , 0.        , 0.3899092 ],
       [0.        , 0.        , 0.28447892],
       [0.3899092 , 0.28447892, 5.78215143]])

In [55]:
a = test_vec[-1]
temp = np.sqrt(a ** 2 + 4 * np.sum(test_vec[:-1] ** 2))
l1 = (a + temp) / 2
l2 = (a - temp) / 2
l1, l2

(5.822163609282079, -0.04001217695906645)

In [56]:
v1 = np.array([*(test_vec[:-1] / l1), 1])
v1 = v1 / np.linalg.norm(v1)

v2 = np.array([*(test_vec[:-1] / l2), 1])
v2 = v2 / np.linalg.norm(v2)

In [57]:
np.allclose(l1 * np.outer(v1, v1) + l2 * np.outer(v2, v2), add_mat)

True

In [58]:
chol1 = np.linalg.cholesky(test_mat)
chol2 = np.linalg.cholesky(test_mat + add_mat)

In [59]:
def cholesky_update(chol, update_vector, multiplier=1.):
    omega = update_vector
    b = 1

    new_chol = np.zeros_like(chol)
    for i in range(chol.shape[0]):
        temp = chol[i, i] ** 2 + multiplier / b * omega[i] ** 2
        new_chol[i, i] = np.sqrt(temp)
        omega[i+1:] -= omega[i] / chol[i, i] * chol[i+1:, i]
        new_chol[i + 1:, i] = new_chol[i, i] * (
                chol[i + 1:, i] / chol[i, i] + multiplier * omega[i] * omega[i + 1:] / temp / b)
        b += multiplier * omega[i] ** 2 / chol[i, i] ** 2
    return new_chol

In [60]:
np.allclose(cholesky_update(cholesky_update(chol1, v1, l1), v2, l2), chol2)

True

In [2]:
## Playing

In [3]:
import numpy as np


In [4]:
y = np.random.rand(10)
B = np.random.rand(10, 3)

In [6]:
rhs1 = B.T @ (y-np.mean(y))
rhs2 = np.zeros_like(rhs1)
for i in range(3):
    rhs2[i] = np.sum(B[:, i] * (y - np.mean(y)))

In [7]:
np.allclose(rhs1, rhs2)

True

In [ ]:
t = np.min()